In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
# given a set of covariates, run linear regression model for each treatment arm
# take argmax of predicted outcome values to determine treatment assignment

covariates = ["attendance_1", "rating_1", "talktime_tutor_pct_1",
    "spoken_token_tutor_pct_1", "chat_token_tutor_pct_1", "length_utterance_tutor_1",
    "length_utterance_student_1", "length_utterance_tutor_chat_1", "length_utterance_student_chat_1",
    "ratio_students_engaged_1", "normalized_num_student_names_used_1", "normalized_num_turns_1",
    "normalized_num_high_uptakes_1", "normalized_num_eliciting_1", "normalized_num_questions_students_1",
    "normalized_num_questions_tutor_1", "normalized_student_reasoning_1", "min_sat_score_series",
    "max_sat_score_series", "grade_for_session_1"]

outcome_vars = ['grade_change', 'uptake_change', 'eliciting_change', 'talktime_change']

In [12]:
arms = ['control', 'tutor', 'tutor_student_social', 'tutor_student_personal']
outcomes = ['uptake', 'eliciting', 'grade', 'talktime']

ridge_weights = {
    'uptake': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'eliciting': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'grade': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'talktime': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    }, # TODO: add an aggreagated outcome measure??
}

lasso_weights = {
    'uptake': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'eliciting': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'grade': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    },
    'talktime': {
        'control': [],
        'tutor': [0.0057869174182523415, -0.04939015234748881, 0.086674115795949, -0.23038033604920116, 
              0.08305298875740905, 7.194821794237663e-05, 0.00042790333804937494, -0.0016273907239995056, 0.0011625396618618778, 
              0.007608148533284885, -0.005157032664444589, -0.006158000152285133, -0.5378618319169195, 0.09970945325939601, 
              0.07182299942295509, 0.18544199470363004, -0.34138781582583033, 0.0022019252435830357, 0.011199830742724497, 
              0.0006215122480511331, 219397820797.08447, -219397820797.0844, -0.03790867036994146],
        'tutor_student_social': [],
        'tutor_student_personal': []
    }, # TODO: add an aggreagated outcome measure??
}
    


In [13]:
summary_df = pd.read_csv('summary_data.csv')
samples_df = summary_df[covariates]
outcome_df = summary_df[outcome_vars]

In [15]:
# loop through each row in samples_df and get predicted policy for each outcome variable

# 1. CREATE SAMPLES MATRIX (num samples x num covariates)
samples = []
for index, row in samples_df.iterrows():
    ndarray = row.to_numpy()  # or row.values
    samples.append(ndarray)

samples_matrix = np.array(samples)

print(samples_matrix.shape)

(629, 20)


In [17]:
# 2. CREATE WEIGHTS MATRIX FOR EACH OUTCOME VAR (num covariates x num arms) 20 x 4

weights = lasso_weights # <--- CHOOSE WHICH ONE YOU'RE DOING!!!
uptake_weights = ndarray()
eliciting_weights = ndarray()
grade_weights = ndarray()
talktime_weights = ndarray()

# uptake
curr_dict = weights['uptake']
uptake_weights = np.array([curr_dict['control'], curr_dict['tutor'], curr_dict['tutor_student_social'], curr_dict['tutor_student_personal']])

# eliciting
curr_dict = weights['eliticing']
eliciting_weights = np.array([curr_dict['control'], curr_dict['tutor'], curr_dict['tutor_student_social'], curr_dict['tutor_student_personal']])

# grade
curr_dict = weights['grade']
grade_weights = np.array([curr_dict['control'], curr_dict['tutor'], curr_dict['tutor_student_social'], curr_dict['tutor_student_personal']])

# talktime
curr_dict = weights['talktime']
talktime_weights = np.array([curr_dict['control'], curr_dict['tutor'], curr_dict['tutor_student_social'], curr_dict['tutor_student_personal']])
                        

1


In [ ]:
# 3. GET SCORES VAR (num samples x num arms) 
# multiply sample matrix by weights matrix for each outcome var

uptake_scores = np.dot(samples_matrix, uptake_weights)
eliciting_scores = np.dot(samples_matrix, eliciting_weights)
grade_scores = np.dot(samples_matrix, grade_weights)
talktime_scores = np.dot(samples_matrix, talktime_weights)

In [ ]:
# 4. GET PREDICTED POLICY (num samples x 1)
# take argmax of scores matrix

uptake_policy = np.argmax(uptake_scores, axis=1)
eliciting_policy = np.argmax(eliciting_scores, axis=1)
grade_policy = np.argmax(grade_scores, axis=1)
talktime_policy = np.argmax(talktime_scores, axis=1)

In [ ]:
# 5. ADD PREDICTED POLICY AS COLUMNS IN SUMMARY_DF

summary_df['uptake_policy'] = uptake_policy
summary_df['eliciting_policy'] = eliciting_policy
summary_df['grade_policy'] = grade_policy
summary_df['talktime_policy'] = talktime_policy

In [ ]:
# 6. EXPORT SUMMARY_DF TO CSV (just so we have it)

summary_df.to_csv('summary_data_with_policy.csv')

## EVALUATION